In [1]:
!nvidia-smi


Wed Oct 25 12:59:35 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   48C    P8    10W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

ROUGE is one of the most popular metrics for evaluating summarization quality. It measures the n-gram overlap between the generated summary and the reference summary. ROUGE is calculated for different n-grams (1-gram, 2-gram, 3-gram), and the scores are combined to obtain an overall score

SacreBLEU provides hassle-free computation of shareable, comparable, and reproducible BLEU scores.

In [2]:
!pip install transformers[sentencepiece] datasets sacrebleu rouge_score py7zr -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 20.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 493.7/493.7 kB 31.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 118.9/118.9 kB 16.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.7/66.7 kB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 38.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 56.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 57.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 61.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 17.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 16.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 78.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 412.3/412.3 kB 31.8 MB/

In [3]:
!pip install --upgrade accelerate
!pip uninstall -y transformers accelerate
!pip install transformers accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.0/261.0 kB 3.2 MB/s eta 0:00:00
Found existing installation: transformers 4.34.1
Uninstalling transformers-4.34.1:
  Successfully uninstalled transformers-4.34.1
Found existing installation: accelerate 0.24.0
Uninstalling accelerate-0.24.0:
  Successfully uninstalled accelerate-0.24.0
  Using cached transformers-4.34.1-py3-none-any.whl (7.7 MB)
  Using cached accelerate-0.24.0-py3-none-any.whl (260 kB)


In [4]:
from transformers import pipeline, set_seed, AutoModelForSeq2SeqLM, AutoTokenizer
from datasets import load_dataset, load_from_disk, load_metric
import matplotlib.pyplot as plt
import pandas as pd
import nltk
from nltk.tokenize import sent_tokenize
from tqdm import tqdm
import torch
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [5]:
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

In [6]:
# The below piece of code is fetched from hugging face "how to use this model from transformers"

model_ckpt = "google/pegasus-cnn_dailymail"

tokenizer = AutoTokenizer.from_pretrained(model_ckpt)
model_pegasus = AutoModelForSeq2SeqLM.from_pretrained(model_ckpt).to(device)

Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-cnn_dailymail and are newly initialized: ['model.decoder.embed_positions.weight', 'model.encoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [7]:
# download and unzip data
# The details about this dataset can be found in below url:
# https://huggingface.co/datasets/samsum

!wget https://github.com/entbappy/Branching-tutorial/raw/master/summarizer-data.zip
!unzip summarizer-data.zip

# From below url we can find the various annotation tools:
# https://newscatcherapi.com/blog/top-6-text-annotation-tools

# The data will be downloaded in csv formats as well as in huggingface formats

--2023-10-25 13:01:32--  https://github.com/entbappy/Branching-tutorial/raw/master/summarizer-data.zip
Resolving github.com (github.com)... 192.30.255.112
Connecting to github.com (github.com)|192.30.255.112|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/entbappy/Branching-tutorial/master/summarizer-data.zip [following]
--2023-10-25 13:01:33--  https://raw.githubusercontent.com/entbappy/Branching-tutorial/master/summarizer-data.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 7903594 (7.5M) [application/zip]
Saving to: ‘summarizer-data.zip’

summarizer-data.zip 100%[===================>]   7.54M  --.-KB/s    in 0.07s   

2023-10-25 13:01:33 (112 MB/s) - ‘summarizer-data.zip’ saved [7903594/

In [8]:
dataset_samsum = load_from_disk('samsum_dataset')
dataset_samsum

DatasetDict({
    train: Dataset({
        features: ['id', 'dialogue', 'summary'],
        num_rows: 14732
    })
    test: Dataset({
        features: ['id', 'dialogue', 'summary'],
        num_rows: 819
    })
    validation: Dataset({
        features: ['id', 'dialogue', 'summary'],
        num_rows: 818
    })
})

In [9]:
for n in dataset_samsum:
  print(n)

print("\n")

for n in dataset_samsum:
  print(dataset_samsum[n])

print("\n")

for n in dataset_samsum:
  print(len(dataset_samsum[n]))

print("\n")

print(f"Features: {dataset_samsum['train'].column_names}")

print("\n")

print(f"Dialogue: {dataset_samsum['train'][1]['dialogue']}")

print("\n")

print(f"Summary: {dataset_samsum['train'][1]['summary']}")

train
test
validation


Dataset({
    features: ['id', 'dialogue', 'summary'],
    num_rows: 14732
})
Dataset({
    features: ['id', 'dialogue', 'summary'],
    num_rows: 819
})
Dataset({
    features: ['id', 'dialogue', 'summary'],
    num_rows: 818
})


14732
819
818


Features: ['id', 'dialogue', 'summary']


Dialogue: Olivia: Who are you voting for in this election? 
Oliver: Liberals as always.
Olivia: Me too!!
Oliver: Great


Summary: Olivia and Olivier are voting for liberals in this election. 


In [10]:
split_lengths = [len(dataset_samsum[split])for split in dataset_samsum]

print(f"Split lengths: {split_lengths}")
print(f"Features: {dataset_samsum['train'].column_names}")
print("\nDialogue")

print(dataset_samsum["test"][1]["dialogue"])

print("\nSummary")

print(dataset_samsum["test"][1]["summary"])

Split lengths: [14732, 819, 818]
Features: ['id', 'dialogue', 'summary']

Dialogue
Eric: MACHINE!
Rob: That's so gr8!
Eric: I know! And shows how Americans see Russian ;)
Rob: And it's really funny!
Eric: I know! I especially like the train part!
Rob: Hahaha! No one talks to the machine like that!
Eric: Is this his only stand-up?
Rob: Idk. I'll check.
Eric: Sure.
Rob: Turns out no! There are some of his stand-ups on youtube.
Eric: Gr8! I'll watch them now!
Rob: Me too!
Eric: MACHINE!
Rob: MACHINE!
Eric: TTYL?
Rob: Sure :)

Summary
Eric and Rob are going to watch a stand-up on youtube.


In [11]:
# Trying to check the value of 'input_id' and 'attention_mask' from tokenizer function for 2 different texts

name1 = "my friend is anoop and he is a good player"
name2 = "my friend Anoop is a good player"
input_encodings = tokenizer(name1, max_length=1024, truncation=True)
print(input_encodings)

target_encodings = tokenizer(name2, max_length=1024, truncation=True)
print(target_encodings)

{'input_ids': [161, 1151, 117, 142, 34909, 111, 178, 117, 114, 234, 1446, 1], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}
{'input_ids': [161, 1151, 983, 34909, 117, 114, 234, 1446, 1], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1]}


In Hugging Face's Transformers library, the **as_target_tokenizer** method is used to convert a tokenizer object into a target tokenizer. Target tokenizers are commonly used in text generation tasks, such as machine translation and text summarization.

When you're working on text generation tasks, you often have both a source text (the input) and a target text (the output or the text to be generated). In these cases, you need two separate tokenizers: one for the source text and another for the target text. The as_target_tokenizer method allows you to create a target tokenizer from an existing tokenizer.

In [12]:
# In this function we are taking input_id and attention mask from the dialogue and input_id from the summary as label

def convert_examples_to_features(example_batch):
    input_encodings = tokenizer(example_batch['dialogue'] , max_length = 1024, truncation = True )

    with tokenizer.as_target_tokenizer():
        target_encodings = tokenizer(example_batch['summary'], max_length = 128, truncation = True )

    return {
        'input_ids' : input_encodings['input_ids'],
        'attention_mask': input_encodings['attention_mask'],
        'labels': target_encodings['input_ids']
    }

In [13]:
dataset_samsum_pt = dataset_samsum.map(convert_examples_to_features, batched = True)

Map:   0%|          | 0/14732 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:3848: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/819 [00:00<?, ? examples/s]

Map:   0%|          | 0/818 [00:00<?, ? examples/s]

In [14]:
print(dataset_samsum_pt['train'])
print(dataset_samsum_pt['test'])
print(dataset_samsum_pt['validation'])

Dataset({
    features: ['id', 'dialogue', 'summary', 'input_ids', 'attention_mask', 'labels'],
    num_rows: 14732
})
Dataset({
    features: ['id', 'dialogue', 'summary', 'input_ids', 'attention_mask', 'labels'],
    num_rows: 819
})
Dataset({
    features: ['id', 'dialogue', 'summary', 'input_ids', 'attention_mask', 'labels'],
    num_rows: 818
})


In [15]:
# Training
# Here below instead of sending the whole data we can send into some batches using DataCollator

from transformers import DataCollatorForSeq2Seq

seq2seq_data_collator = DataCollatorForSeq2Seq(tokenizer, model=model_pegasus)

In [16]:
from transformers import TrainingArguments, Trainer

# Below we are providing the training arguments
trainer_args = TrainingArguments(
    output_dir = 'pegasus-samsum', num_train_epochs=1, warmup_steps=500,
    per_device_train_batch_size=1, per_device_eval_batch_size=1,
    weight_decay=0.01, logging_steps=500, save_steps=1e6,
    gradient_accumulation_steps=16
)

In [17]:
# Here we are performing the training, also we are taking test data for my training because train data size is huge

trainer = Trainer(model=model_pegasus, args=trainer_args, tokenizer=tokenizer, data_collator=seq2seq_data_collator,
                  train_dataset=dataset_samsum_pt['test'], eval_dataset=dataset_samsum_pt['validation'])

In [18]:
trainer.train()

You're using a PegasusTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss


TrainOutput(global_step=51, training_loss=3.075412525850184, metrics={'train_runtime': 149.5703, 'train_samples_per_second': 5.476, 'train_steps_per_second': 0.341, 'total_flos': 313317832187904.0, 'train_loss': 3.075412525850184, 'epoch': 1.0})

In [2]:
# function to generate different chunks of data of given batch size
def generate_batch_sized_chunks(list_of_elements, batch_size):
    """split the dataset into smaller batches that we can process simultaneously
    Yield successive batch-sized chunks from list_of_elements."""
    for i in range(0, len(list_of_elements), batch_size):
        yield list_of_elements[i : i + batch_size]


# function to generate different chunks of data of given batch size
def calculate_metric_on_test_ds(dataset, batch_size=16, column_text="dialogue", column_summary="summary"):
    """create the batches of data for dialogue and summary for the given batch size
    """
    dialogue_batches = list(generate_batch_sized_chunks(dataset[column_text], batch_size))
    summary_batches = list(generate_batch_sized_chunks(dataset[column_summary], batch_size))
    return dialogue_batches, summary_batches

#a, b = calculate_metric_on_test_ds(dataset_samsum['test'][0:10], batch_size = 2, column_text = 'dialogue', column_summary= 'summary')

In [20]:
# Here as we gave the batch size of 2 which means total 5 batches will get created for total of 10 data points and each batch has 2 data points

dialogue_batches, summary_batches = calculate_metric_on_test_ds(dataset_samsum['test'][0:10], batch_size = 1, column_text = 'dialogue', column_summary = 'summary')

for dialogue_batch, summary_batch in tqdm(zip(dialogue_batches, summary_batches), total=len(dialogue_batches)):
    print("\nDialogue batch")
    print(dialogue_batch)
    print("\nSummary batch")
    print(summary_batch)

100%|██████████| 10/10 [00:00<00:00, 1296.58it/s]


Dialogue batch
["Hannah: Hey, do you have Betty's number?\nAmanda: Lemme check\nHannah: <file_gif>\nAmanda: Sorry, can't find it.\nAmanda: Ask Larry\nAmanda: He called her last time we were at the park together\nHannah: I don't know him well\nHannah: <file_gif>\nAmanda: Don't be shy, he's very nice\nHannah: If you say so..\nHannah: I'd rather you texted him\nAmanda: Just text him 🙂\nHannah: Urgh.. Alright\nHannah: Bye\nAmanda: Bye bye"]

Summary batch
["Hannah needs Betty's number but Amanda doesn't have it. She needs to contact Larry."]

Dialogue batch
["Eric: MACHINE!\r\nRob: That's so gr8!\r\nEric: I know! And shows how Americans see Russian ;)\r\nRob: And it's really funny!\r\nEric: I know! I especially like the train part!\r\nRob: Hahaha! No one talks to the machine like that!\r\nEric: Is this his only stand-up?\r\nRob: Idk. I'll check.\r\nEric: Sure.\r\nRob: Turns out no! There are some of his stand-ups on youtube.\r\nEric: Gr8! I'll watch them now!\r\nRob: Me too!\r\nEric: MACH

In Hugging Face's Transformers library, the tokenizer function performs several key tasks when working with natural language processing (NLP) models. These tasks include:

**Text Tokenization**: Tokenization is the process of splitting a text into smaller units called tokens. Tokens can be words, subwords, or characters, depending on the specific tokenizer used. The tokenizer function takes a text input and converts it into a list of tokens. For example, it can split the sentence "I love NLP" into the tokens ["I", "love", "NLP"].

**Numerical Encoding**: Once the text is tokenized, the tokenizer encodes these tokens into numerical values. This is essential because most NLP models operate on numerical inputs. The numerical encoding assigns an integer or a unique identifier to each token. For example, "I" might be assigned the integer 34, "love" could be 56, and "NLP" might be 112.

**Padding and Truncation**: Texts often have varying lengths, but models typically require fixed-length inputs. The tokenizer can add padding tokens to make all inputs the same length or truncate tokens from longer inputs.

**Special Tokens**: Some tokens are reserved for special purposes, like [CLS] and [SEP] tokens used in BERT for classification tasks. The tokenizer adds these special tokens as needed.

**Attention Masks**: The tokenizer also generates an attention mask that helps the model identify which tokens are actual content and which are padding. This is used to prevent the model from paying attention to padding tokens during training.

**Subword Tokenization**: Many modern NLP models, like BERT and GPT-2, use subword tokenization. Words are split into smaller units, which can be useful for handling out-of-vocabulary words and reducing the vocabulary size.

**Decoding**: The tokenizer also provides a decoding function, which converts numerical token IDs back into human-readable text.

In [21]:
#dialogue_batches[0]
inputs = tokenizer(dialogue_batches[0], max_length=1024,  truncation=True, padding="max_length", return_tensors="pt")
print(len(dialogue_batches[0]), dialogue_batches[0])
print("\n")
print(len(inputs), inputs)

1 ["Hannah: Hey, do you have Betty's number?\nAmanda: Lemme check\nHannah: <file_gif>\nAmanda: Sorry, can't find it.\nAmanda: Ask Larry\nAmanda: He called her last time we were at the park together\nHannah: I don't know him well\nHannah: <file_gif>\nAmanda: Don't be shy, he's very nice\nHannah: If you say so..\nHannah: I'd rather you texted him\nAmanda: Just text him 🙂\nHannah: Urgh.. Alright\nHannah: Bye\nAmanda: Bye bye"]


2 {'input_ids': tensor([[12636,   151, 10532,  ...,     0,     0,     0]]), 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0]])}


In Hugging Face's Transformers library, the **model.generate** method is used for text generation with pre-trained language models. It allows you to generate text based on a given input prompt or a set of input tokens. This method is particularly useful for tasks like text completion, language generation, and text summarization.

In [22]:
summaries = trainer.model.generate(input_ids=inputs["input_ids"].to(device),
                         attention_mask=inputs["attention_mask"].to(device),
                         length_penalty=0.8, num_beams=8, max_length=128)
print(summaries)

tensor([[    0, 12636,   151,   125,   131,   252,   880,   119, 50496,   342,
           110,   107,   106, 41985,   151,   125,   131,   252,   880,   119,
         50496,   342,   110,   107,   106, 48522,   151,   125,   131,   252,
           880,   119, 50496,   342,   110,   107,     1]], device='cuda:0')


In [23]:
# Below we see that the original dialogue and summary of the 1st input

print(dialogue_batches[0])
print("\n")
print(summary_batches[0])
print("\n")
print(tokenizer.decode([    0, 12195,  6937, 12636,   175,   265,   148, 17379,   131,   116,
           344,   110,   107,   106, 48522,   151,   125,   131,   252,   880,
           119, 50496,   342,   110,   107,   106, 48522,   151,   125,   131,
           252,   880,   119, 50496,   342, 12195,   151,  1205,  1352,   342,
           110,   107,     1], skip_special_tokens=True, clean_up_tokenization_spaces=True))

["Hannah: Hey, do you have Betty's number?\nAmanda: Lemme check\nHannah: <file_gif>\nAmanda: Sorry, can't find it.\nAmanda: Ask Larry\nAmanda: He called her last time we were at the park together\nHannah: I don't know him well\nHannah: <file_gif>\nAmanda: Don't be shy, he's very nice\nHannah: If you say so..\nHannah: I'd rather you texted him\nAmanda: Just text him 🙂\nHannah: Urgh.. Alright\nHannah: Bye\nAmanda: Bye bye"]


["Hannah needs Betty's number but Amanda doesn't have it. She needs to contact Larry."]


Amanda asks Hannah if she has Betty's number.<n>Hannah: I'd rather you texted him.<n>Hannah: I'd rather you texted him Amanda: Just text him.


In [24]:
for s in summaries:
  print(1)

1


In [25]:
decoded_summaries = [tokenizer.decode(s, skip_special_tokens=True, clean_up_tokenization_spaces=True) for s in summaries]
print(decoded_summaries)

["Hannah: I'd rather you texted him.<n>Amanda: I'd rather you texted him.<n>Hannah: I'd rather you texted him."]


In [33]:
rouge_metric = load_metric('rouge')
rouge_metric.add_batch(predictions=decoded_summaries, references=summary_batches[0])
rouge_metric.compute()

{'rouge1': AggregateScore(low=Score(precision=0.08695652173913043, recall=0.125, fmeasure=0.10256410256410256), mid=Score(precision=0.08695652173913043, recall=0.125, fmeasure=0.10256410256410256), high=Score(precision=0.08695652173913043, recall=0.125, fmeasure=0.10256410256410256)),
 'rouge2': AggregateScore(low=Score(precision=0.0, recall=0.0, fmeasure=0.0), mid=Score(precision=0.0, recall=0.0, fmeasure=0.0), high=Score(precision=0.0, recall=0.0, fmeasure=0.0)),
 'rougeL': AggregateScore(low=Score(precision=0.08695652173913043, recall=0.125, fmeasure=0.10256410256410256), mid=Score(precision=0.08695652173913043, recall=0.125, fmeasure=0.10256410256410256), high=Score(precision=0.08695652173913043, recall=0.125, fmeasure=0.10256410256410256)),
 'rougeLsum': AggregateScore(low=Score(precision=0.08695652173913043, recall=0.125, fmeasure=0.10256410256410256), mid=Score(precision=0.08695652173913043, recall=0.125, fmeasure=0.10256410256410256), high=Score(precision=0.08695652173913043, r

**ROUGE Precision (ROUGE-P):**

ROUGE-P measures the precision of the generated text concerning the reference text.
Precision is a measure of how many of the generated words or phrases are relevant and correctly matched to the reference. A high ROUGE-P score indicates that the generated text is precise and has minimal irrelevant content.

**ROUGE Recall (ROUGE-R):**

ROUGE-R measures the recall of the generated text concerning the reference text.
Recall is a measure of how many of the reference words or phrases are correctly included in the generated text. A high ROUGE-R score indicates that the generated text captures most of the important information from the reference.

**ROUGE F1 Score (ROUGE-F1):**

ROUGE-F1 is the harmonic mean of precision and recall.
It provides a balanced evaluation of the quality of the generated text, considering both precision and recall. A high ROUGE-F1 score indicates a good balance between precision and recall.
Now, regarding "low," "mid," and "high" ROUGE:

These terms do not refer to specific metrics but rather to different ROUGE scores that are based on the n-gram overlap between the generated text and the reference text. ROUGE calculates scores for different n-gram lengths, including 1-gram (unigrams), 2-gram (bigrams), and sometimes longer n-grams. Here's a breakdown:

**Low ROUGE**: Typically refers to ROUGE scores based on 1-gram overlaps (unigrams). Low ROUGE scores mean that the generated text has a limited overlap with the reference, and it may not capture important details.

**Mid ROUGE**: Refers to ROUGE scores based on 2-gram (bigram) overlaps, which consider pairs of adjacent words. Mid ROUGE scores provide a slightly more in-depth evaluation, considering the presence of two-word phrases in the generated text.

**High ROUGE**: Refers to ROUGE scores that consider longer n-grams (e.g., trigrams, 4-grams, etc.). High ROUGE scores suggest that the generated text captures longer and more meaningful phrases from the reference text.

In summary, ROUGE metrics, including precision, recall, and F1 score, are used to assess the quality of generated text, with "low," "mid," and "high" ROUGE indicating the level of n-gram overlap being evaluated (from single words to longer phrases) to provide a more detailed understanding of the quality of generated content.


The 'max_length' parameter is used to specify the maximum number of tokens that the model should generate.
The 'length_penalty' parameter is a hyperparameter that can be set to a positive value, where higher values encourage the model to
generate longer sequences, and lower values encourage shorter sequences.
If you set length_penalty to a value less than 1, it will encourage the model to produce shorter output.
If you set it to a value greater than 1, it will encourage longer output. A value of 1 effectively disables the length penalty

' represents the number of "beams" or parallel sequences that the model considers during text generation. In beam search, instead of generating a single sequence, the model generates multiple sequences in parallel, each starting from the same input but with slight variations. The model then scores these generated sequences and selects the top num_beams sequences with the highest scores.

By using a larger num_beams value, you can encourage the model to explore a broader range of possibilities and potentially generate more diverse and contextually relevant text. However, increasing num_beams also increases computational overhead.

The choice of the num_beams value depends on your specific use case and the trade-off between computational cost and the quality of generated text. Common values for num_beams might range from 1 (no beam search, equivalent to greedy decoding) to higher values like 5, 10, or even more, depending on your requirements.

In Hugging Face's Transformers library, the **.to(device)** method is used to move a PyTorch tensor or model to a specific device, such as a CPU or a GPU. This is particularly important when working with deep learning models on GPUs, as GPUs are much faster at performing matrix operations and can significantly accelerate the training and inference processes.

In [38]:
def compute_metric(metric, model, tokenizer, device=device):

    article_batches, target_batches = calculate_metric_on_test_ds(dataset_samsum['test'][0:10], batch_size = 2,
                                                                  column_text = 'dialogue', column_summary= 'summary')

    for dialogue_batch, summary_batch in tqdm(
        zip(dialogue_batches, summary_batches), total=len(dialogue_batches)):

        # here we are getting input_id and attention_mask
        inputs = tokenizer(dialogue_batch, max_length=1024,  truncation=True,
                        padding="max_length", return_tensors="pt")

        # here we are generating summaries based on the inputs
        summaries = model.generate(input_ids=inputs["input_ids"].to(device),
                         attention_mask=inputs["attention_mask"].to(device),
                         length_penalty=0.8, num_beams=8, max_length=128)
        ''' parameter for length penalty ensures that the model does not generate sequences that are too long. '''

        # Finally, we decode the generated texts,
        # replace the  token, and add the decoded texts with the references to the metric.
        decoded_summaries = [tokenizer.decode(s, skip_special_tokens=True, clean_up_tokenization_spaces=True) for s in summaries]
        decoded_summaries = [d.replace("", " ") for d in decoded_summaries]

        metric.add_batch(predictions=decoded_summaries, references=summary_batch)

    #  Finally compute and return the ROUGE scores.
    score = metric.compute()
    return score


Please read article in below link so as to know how to evaluate a text summarization model:

https://stackoverflow.com/questions/9879276/how-do-i-evaluate-a-text-summarization-tool

In [39]:
rouge_names = ["rouge1", "rouge2", "rougeL", "rougeLsum"]
rouge_metric = load_metric('rouge')

In [41]:
''' Here we can see that we are getting below low, mid and high for below rouge score type:
 - rouge1
 - rouge2
 - rougeL
 - rougeLsum
'''

score = compute_metric(rouge_metric, trainer.model, tokenizer)
score

100%|██████████| 10/10 [00:21<00:00,  2.19s/it]


{'rouge1': AggregateScore(low=Score(precision=0.006023017278629223, recall=0.039598581918241206, fmeasure=0.010129309329647587), mid=Score(precision=0.011593985473623709, recall=0.06851179673321234, fmeasure=0.019098515143192867), high=Score(precision=0.017233575138873653, recall=0.09704330807471767, fmeasure=0.028169293914334037)),
 'rouge2': AggregateScore(low=Score(precision=0.0, recall=0.0, fmeasure=0.0), mid=Score(precision=0.0, recall=0.0, fmeasure=0.0), high=Score(precision=0.0, recall=0.0, fmeasure=0.0)),
 'rougeL': AggregateScore(low=Score(precision=0.006195686132385056, recall=0.0408608882914651, fmeasure=0.010569373566395075), mid=Score(precision=0.011583011890625782, recall=0.06886940101523523, fmeasure=0.019074718640469744), high=Score(precision=0.017648474879806406, recall=0.09736086584829486, fmeasure=0.028361553444469584)),
 'rougeLsum': AggregateScore(low=Score(precision=0.006154479095292823, recall=0.03619301873786981, fmeasure=0.010139069983867162), mid=Score(precisi

In [46]:
for rn in rouge_names:
  print(rn, score[rn].mid.fmeasure)

rouge1 0.019098515143192867
rouge2 0.0
rougeL 0.019074718640469744
rougeLsum 0.018950774854909633


In [47]:
'''Here we can see that the metrics which was calculated above for low, mid and high is captured as a dataframe for below types:
  - rouge1
  - rouge2
  - rougeL
  - rougeLsum
'''

score = compute_metric(rouge_metric, trainer.model, tokenizer)

rouge_dict = dict((rn, score[rn].mid.fmeasure ) for rn in rouge_names )

pd.DataFrame(rouge_dict, index = [f'pegasus'] )

100%|██████████| 10/10 [00:35<00:00,  3.57s/it]


,rouge1,rouge2,rougeL,rougeLsum
pegasus,0.016837,0.0,0.016768,0.016971


Now we will be saving tokenizer as well as model as during prediction we need both these things

In [48]:
# Save model

model_pegasus.save_pretrained("pegasus-samsum-model")

In [49]:
# Save tokenizer

tokenizer.save_pretrained("tokenizer")

('tokenizer/tokenizer_config.json',
 'tokenizer/special_tokens_map.json',
 'tokenizer/spiece.model',
 'tokenizer/added_tokens.json',
 'tokenizer/tokenizer.json')

In [50]:
# Load

tokenizer = AutoTokenizer.from_pretrained("/content/tokenizer")

In [60]:
# Prediction

gen_kwargs = {"length_penalty": 0.8, "num_beams":8, "max_length": 128}

sample_text = dataset_samsum["test"][0]["dialogue"]

reference = dataset_samsum["test"][0]["summary"]

pipe = pipeline("summarization", model="pegasus-samsum-model", tokenizer=tokenizer)

##
print("Dialogue:")
print(sample_text)

print("\nReference Summary:")
print(reference)

print("\nModel Summary:")
result = pipe(sample_text, **gen_kwargs)
print(result[0]['summary_text'])

Your max_length is set to 128, but your input_length is only 122. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=61)


Dialogue:
Hannah: Hey, do you have Betty's number?
Amanda: Lemme check
Hannah: <file_gif>
Amanda: Sorry, can't find it.
Amanda: Ask Larry
Amanda: He called her last time we were at the park together
Hannah: I don't know him well
Hannah: <file_gif>
Amanda: Don't be shy, he's very nice
Hannah: If you say so..
Hannah: I'd rather you texted him
Amanda: Just text him 🙂
Hannah: Urgh.. Alright
Hannah: Bye
Amanda: Bye bye

Reference Summary:
Hannah needs Betty's number but Amanda doesn't have it. She needs to contact Larry.

Model Summary:
Amanda: Ask Larry Amanda: He called her last time we were at the park together .<n>Hannah: I'd rather you texted him .<n>Amanda: Just text him .


the **pipeline** function is a high-level API that simplifies the process of using pre-trained models for various natural language processing (NLP) tasks. It provides an easy and user-friendly way to load, fine-tune, and use models for specific tasks without requiring extensive knowledge of the library's internals. The pipeline function streamlines common NLP tasks, making them more accessible and efficient.

Here's how the pipeline function works:

**Task Specification**: When you create a pipeline using pipeline(task, model, framework), you specify the NLP task you want to perform (e.g., "sentiment-analysis," "text-generation," "ner" for named entity recognition, etc.). You also specify the model name or directory path (model) for the pre-trained model you want to use. The optional framework parameter allows you to specify the deep learning framework (e.g., "pt" for PyTorch, "tf" for TensorFlow).

**Automatic Loading**: The pipeline function automatically loads the pre-trained model, its associated tokenizer, and any necessary configurations for the specified task. You don't need to manually load or set up the model components.

**Tokenization**: The input text provided to the pipeline is tokenized and formatted based on the model's tokenizer. This includes breaking text into subword tokens, adding special tokens, and generating input IDs.

**Model Inference**: The pipeline takes care of sending the tokenized input to the model for inference, receiving model predictions, and decoding the model's output.

**Post-processing**: After model inference, the pipeline may perform post-processing tasks specific to the chosen task. For instance, it can map model output IDs to labels, remove padding tokens, and return the results in a human-readable format.

User-Friendly Output: The pipeline function returns the results of the NLP task in an easy-to-use format, often as a list of dictionaries, where each dictionary contains information related to the task, such as labels, scores, or other relevant details.

The pipeline function simplifies and accelerates the process of using pre-trained NLP models for specific tasks, making it accessible to users with various levels of expertise. It is particularly useful for tasks where pre-trained models are available and can be fine-tuned for specific applications or when you need to perform common NLP tasks quickly and efficiently.